In [1]:
import pandas as pd
import nltk
import emoji
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import re
from sklearn.pipeline import Pipeline

In [2]:
train=pd.read_csv(r'C:\Users\alyss\Desktop/train.csv')
test=pd.read_csv(r'C:\Users\alyss\Desktop/test.csv')
eva=pd.read_csv(r'C:\Users\alyss\Desktop/evaluation.csv')

In [3]:
print(train.shape)
print(test.shape)
print(eva.shape)

(7500, 2)
(2500, 2)
(5000, 2)


In [5]:
train.isnull().sum()##check null values

score    0
text     0
dtype: int64

In [6]:
test.isnull().sum()

score    0
text     0
dtype: int64

In [7]:
eva.isnull().sum()

score    0
text     0
dtype: int64

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [10]:
##ask chatgpt how to remove emoji
def remove_emoji(text):
    return emoji.replace_emoji(text,replace='')

In [11]:
##https://blog.csdn.net/WHYbeHERE/article/details/115512635
def data_process(text):
    text = re.sub(r'<[^>]+>', '', text)
    text=re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'(.)\1+', r'\1', text)
    text=text.lower()
    words=text.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [13]:
train['cleantext']=train['text'].apply(data_process)
test['cleantext']=test['text'].apply(data_process)
eva['cleantext']=eva['text'].apply(data_process)

In [14]:
train['cleantext']=train['text'].apply(remove_emoji)
test['cleantext']=test['text'].apply(remove_emoji)
eva['cleantext']=eva['text'].apply(remove_emoji)

In [15]:
mapping = {
    0: 'negative',
    1: 'positive',
}

train['score'] = train['score'].map(mapping)
test['score'] = test['score'].map(mapping)
eva['score'] = eva['score'].map(mapping)

In [16]:
print(train.shape)
print(test.shape)
print(eva.shape)

(7500, 3)
(2500, 3)
(5000, 3)


In [19]:
x_train=train['cleantext']
y_train=train['score']
x_test=test['cleantext']
y_test=test['score']
x_eva=eva['cleantext']
y_eva=eva['score']

In [20]:
##https://www.capitalone.com/tech/machine-learning/understanding-tf-idf/
vectorizer=TfidfVectorizer()

In [21]:
x_train_tfidf=vectorizer.fit_transform(x_train)
x_test_tfidf=vectorizer.transform(x_test)
x_eva_tfidf=vectorizer.transform(x_eva)

In [22]:
print(x_eva_tfidf.shape)
print(x_test_tfidf.shape)
print(x_train_tfidf.shape)

(5000, 35275)
(2500, 35275)
(7500, 35275)


In [23]:
nb=MultinomialNB()

In [24]:
nb.fit(x_train_tfidf,y_train)

MultinomialNB()

In [25]:
svm=SVC(kernel='linear')

In [26]:
svm.fit(x_train_tfidf,y_train)

SVC(kernel='linear')

In [27]:
nb_pred=nb.predict(x_test_tfidf)

In [28]:
svm_pred=svm.predict(x_test_tfidf)

In [29]:
print("Naive Bayes classification performance:")
print(classification_report(y_test,nb_pred,digits=4))

Naive Bayes classification performance:
              precision    recall  f1-score   support

    negative     0.7910    0.9217    0.8513      1252
    positive     0.9059    0.7556    0.8239      1248

    accuracy                         0.8388      2500
   macro avg     0.8484    0.8387    0.8376      2500
weighted avg     0.8483    0.8388    0.8377      2500



In [30]:
print("SVM classification performance:")
print(classification_report(y_test,svm_pred,digits=4))

SVM classification performance:
              precision    recall  f1-score   support

    negative     0.8480    0.8602    0.8541      1252
    positive     0.8577    0.8454    0.8515      1248

    accuracy                         0.8528      2500
   macro avg     0.8529    0.8528    0.8528      2500
weighted avg     0.8529    0.8528    0.8528      2500



In [31]:
nb_eva_pred=nb.predict(x_eva_tfidf)

In [33]:
print("Naive Bayes classification of evaluation:")
print(classification_report(y_eva,nb_eva_pred,digits=4))

Naive Bayes classification of evaluation:
              precision    recall  f1-score   support

    negative     0.7357    0.9456    0.8276      2482
    positive     0.9254    0.6652    0.7740      2518

    accuracy                         0.8044      5000
   macro avg     0.8306    0.8054    0.8008      5000
weighted avg     0.8313    0.8044    0.8006      5000



In [36]:
svm_eva_pred=svm.predict(x_eva_tfidf)

In [37]:
print("SVM classification of evaluation:")
print(classification_report(y_eva,svm_eva_pred,digits=4))

SVM classification of evaluation:
              precision    recall  f1-score   support

    negative     0.8465    0.8578    0.8521      2482
    positive     0.8579    0.8467    0.8523      2518

    accuracy                         0.8522      5000
   macro avg     0.8522    0.8522    0.8522      5000
weighted avg     0.8523    0.8522    0.8522      5000

